## ServiceCore Data Test by Yaron Shamash 
- The task at hand is modeled on part of a common data ingestion workflow. A customer provides
us with two sources of data. One is their customer sheet, which has come from QuickBooks.
The second is their route sheet, which they have created in Excel. In order to ingest their data
into our system, we need to match the customer names in the route sheet against the customer
names in the customer sheet, and parse the data into a JSON object that can be passed to our
API.


In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import re
from numpy import int64
import json
from pandas.io.json import json_normalize 
%matplotlib inline

In [2]:
# read datasets and rename the customer sheet columns 
dfc = pd.read_csv('Customers - ServiceCore Data Test.csv')
dfr = pd.read_csv('Routesheet - ServiceCore Data Test.csv')
dfc.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)

In [3]:
# dimensions and view of routesheet df
print(dfr.shape)
dfr.head()

(2528, 8)


,Customer,TOILET #,Job Name/Well Name,Address,City,Schedule,Scheduled Day,Charge Type
0,src,1,Ag Pad FRAC,O Street x 59th Ave W1.1 S into,West Greeley,2xWeekly,7/22/18change billing to production,pending
1,Hauer Custom Homes,3611,NaN,"19299 CR 70, Eaton",Eaton,Weekly,address does not exist 10/7/19,MONTHLY (8/1/19)
2,Ridgeway Custom homes,1,NaN,"6879 Crooked Stick, Windsor",West Windsor,Weekly,address does not exist 3/22/19,MONTHLY (9/19/18)
3,ASTER RIDGE,3051,NaN,1827 AA ST,EAST GREELEY,WEEKLY,BISON RIDGE TOOK OVER TOILET 1/27/22,MONTHLY (10/12/21)
4,Blackeagle,"3540, 2489",Angus Compressor Station,"60315 CR 71, Grover\n128x69 E1 N into",Grover,Weekly,BLOWNOVER/TRADED 10/24/19,MONTHLY (6/11/19)


In [4]:
# dimensions and view of customers df
print(dfc.shape)
dfc.head()

(15135, 21)


,customer,company,mr./ms./...,first_name,m.i.,last_name,main_phone,main_email,bill_to_1,bill_to_2,...,bill_to_4,bill_to_5,ship_to_1,ship_to_2,ship_to_3,ship_to_4,ship_to_5,terms,sales_tax_code,tax_item
0,*JESUS SANCHEZ,JESUS SANCHEZ,NaN,NaN,NaN,NaN,9670-342-6100,NaN,JESUS SANCHEZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tax,NON TAXABLE ITEM
1,*JESUS SANCHEZ:17900 CR 5,JESUS SANCHEZ,NaN,NaN,NaN,NaN,9670-342-6100,NaN,JESUS SANCHEZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tax,NON TAXABLE ITEM
2,1888 INDUSTRIAL SERVICES,1888 INDUSTRIAL SERVICES,NaN,NaN,NaN,NaN,970-702-7610,AP@1888IS.COM,1888 INDUSTRIAL SERVICES,"800 8TH AVE, SUITE 301",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
3,1888 INDUSTRIAL SERVICES:WELLS RANCH TO REPUBLIC,1888 INDUSTRIAL SERVICES,NaN,JOSEPH,NaN,MONTOYA,970-702-7610,AP@1888IS.COM,1888 INDUSTRIAL SERVICES,"800 8TH AVE, SUITE 301",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Tax,Wyoming Sales Tax
4,2 RINGS TRUCKING,2 RINGS TRUCKING,NaN,NaN,NaN,NaN,406-289-0901,NaN,2 RINGS TRUCKING,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Due on receipt,Non,NON TAXABLE ITEM


In [5]:
# check data types and nulls
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15135 entries, 0 to 15134
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   customer        15135 non-null  object 
 1   company         14957 non-null  object 
 2   mr./ms./...     11 non-null     object 
 3   first_name      2173 non-null   object 
 4   m.i.            67 non-null     object 
 5   last_name       2033 non-null   object 
 6   main_phone      13135 non-null  object 
 7   main_email      12892 non-null  object 
 8   bill_to_1       15113 non-null  object 
 9   bill_to_2       9797 non-null   object 
 10  bill_to_3       9619 non-null   object 
 11  bill_to_4       38 non-null     object 
 12  bill_to_5       0 non-null      float64
 13  ship_to_1       35 non-null     object 
 14  ship_to_2       32 non-null     object 
 15  ship_to_3       29 non-null     object 
 16  ship_to_4       3 non-null      object 
 17  ship_to_5       0 non-null     

In [6]:
# customer df sum nulls columns. 0 nulls in the customer column 
dfc.isnull().sum()

customer              0
company             178
mr./ms./...       15124
first_name        12962
m.i.              15068
last_name         13102
main_phone         2000
main_email         2243
bill_to_1            22
bill_to_2          5338
bill_to_3          5516
bill_to_4         15097
bill_to_5         15135
ship_to_1         15100
ship_to_2         15103
ship_to_3         15106
ship_to_4         15132
ship_to_5         15135
terms              3560
sales_tax_code     3202
tax_item            599
dtype: int64

Using the two source files provided, implement the following logic:
-Match the route sheet against the customer sheet based on the “Customer” field in both
tables.
- Note: matching is case-insensitive.
- Only the top level customer from the QuickBooks customers export should be
matched against. For example, the top level customer in the QuickBooks
customer field “John Smith:123 Main Street” would be “John Smith”.
- For each unique customer name (case insensitive) in the route sheet, create an object
consisting of the following fields:
○ From the customer sheet:
- Customer
- Bill to 1
- Bill to 2
- Bill to 3
- Main Phone
- Note: each phone number should be formatted to include only
numeric characters. For example “555-123-4567: Tracy” should
become “5551234567”.
- Main Email
- Terms

### The customer column contains colons, apostrophes and slashes which should be removed.  There are also text strings such as LLC and INC which should be removed for a better match on the Route sheet.

In [7]:
# remove white space, text after the colons, asterisks, commas and slashes from the customer column, 'LLC', 'INC'
dfc.customer.str.strip()
dfc.customer=dfc.customer.str.split(':').str[0]
dfc.customer=dfc.customer.str.split(',').str[0]
dfc.customer=dfc.customer.str.split('/').str[0]
dfc.customer=dfc.replace({'\*': ''}, regex=True)
dfc.customer.str.replace('LLC/','')
dfc.customer.str.replace('INC/','')

0                   JESUS SANCHEZ
1                   JESUS SANCHEZ
2        1888 INDUSTRIAL SERVICES
3        1888 INDUSTRIAL SERVICES
4                2 RINGS TRUCKING
                   ...           
15130                       ZTERS
15131                       ZTERS
15132                       ZTERS
15133                       ZTERS
15134                       ZTERS
Name: customer, Length: 15135, dtype: object

In [8]:
# duplicate rows in the customer column
dfc[dfc["customer"].duplicated()]

,customer,company,mr./ms./...,first_name,m.i.,last_name,main_phone,main_email,bill_to_1,bill_to_2,...,bill_to_4,bill_to_5,ship_to_1,ship_to_2,ship_to_3,ship_to_4,ship_to_5,terms,sales_tax_code,tax_item
1,JESUS SANCHEZ,JESUS SANCHEZ,NaN,NaN,NaN,NaN,9670-342-6100,NaN,JESUS SANCHEZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tax,NON TAXABLE ITEM
3,1888 INDUSTRIAL SERVICES,1888 INDUSTRIAL SERVICES,NaN,JOSEPH,NaN,MONTOYA,970-702-7610,AP@1888IS.COM,1888 INDUSTRIAL SERVICES,"800 8TH AVE, SUITE 301",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Tax,Wyoming Sales Tax
5,2 RINGS TRUCKING,2 RINGS TRUCKING,NaN,NaN,NaN,NaN,406-289-0901,NaN,2 RINGS TRUCKING,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non,NON TAXABLE ITEM
7,2 VALLEY BUILDERS,"2 VALLEY BUILDERS, INC",NaN,NaN,NaN,NaN,970-599-2134- John,2valleybuilders@gmail.com,"2 VALLEY BUILDERS, INC",6637 SPANISH BAY DRIVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
8,2 VALLEY BUILDERS,"2 VALLEY BUILDERS, INC",NaN,NaN,NaN,NaN,970-599-2134- John,2valleybuilders@gmail.com,"2 VALLEY BUILDERS, INC",6637 SPANISH BAY DRIVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15130,ZTERS,ZTERS INC.,NaN,NaN,NaN,NaN,832-698-2203 X 106 Mary Alvarado/AP,Invoices@zters.com,ZTERS INC.,13727 Office Park Drive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
15131,ZTERS,ZTERS INC.,NaN,NaN,NaN,NaN,832-698-2203 X 106 Mary Alvarado/AP,Invoices@zters.com,ZTERS INC.,13727 Office Park Drive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tax,NON TAXABLE ITEM
15132,ZTERS,ZTERS INC.,NaN,NaN,NaN,NaN,832-698-2203 X 106 Mary Alvarado/AP,Invoices@zters.com,ZTERS INC.,13727 Office Park Drive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Tax,NON TAXABLE ITEM
15133,ZTERS,ZTERS INC.,NaN,NaN,NaN,NaN,832-698-2203 X 106 Mary Alvarado/AP,Invoices@zters.com,ZTERS INC.,13727 Office Park Drive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM


In [9]:
# drop duplicate rows in company column and store in new df
dfcl=dfc.drop_duplicates(subset='customer', keep='first', inplace=False)
dfcl

,customer,company,mr./ms./...,first_name,m.i.,last_name,main_phone,main_email,bill_to_1,bill_to_2,...,bill_to_4,bill_to_5,ship_to_1,ship_to_2,ship_to_3,ship_to_4,ship_to_5,terms,sales_tax_code,tax_item
0,JESUS SANCHEZ,JESUS SANCHEZ,NaN,NaN,NaN,NaN,9670-342-6100,NaN,JESUS SANCHEZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tax,NON TAXABLE ITEM
2,1888 INDUSTRIAL SERVICES,1888 INDUSTRIAL SERVICES,NaN,NaN,NaN,NaN,970-702-7610,AP@1888IS.COM,1888 INDUSTRIAL SERVICES,"800 8TH AVE, SUITE 301",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
4,2 RINGS TRUCKING,2 RINGS TRUCKING,NaN,NaN,NaN,NaN,406-289-0901,NaN,2 RINGS TRUCKING,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Due on receipt,Non,NON TAXABLE ITEM
6,2 VALLEY BUILDERS,"2 VALLEY BUILDERS, INC",NaN,NaN,NaN,NaN,970-599-2134- John,2valleybuilders@gmail.com,"2 VALLEY BUILDERS, INC",6637 SPANISH BAY DRIVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
15,2020 APEX LLC,2020 APEX LLC,NaN,NaN,NaN,NaN,970.381.1081 Ryan Andre,RANDRE@SEARSREALESTATE.COM,2020 APEX LLC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15053,ZAK DIRT,"ZAK DIRT, INC.",NaN,BRANDI,NaN,WILSON,970-535-4657,BWILSON@ZAKDIRT.COM; accounting@zakdirt.com,"ZAK DIRT, INC.",14290 HILLTOP ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
15060,ZAP Engineering & Construction Services,ZAP Engineering & Construction Services,NaN,NaN,NaN,NaN,303-565-5567,apinvoices@zapecs.com,ZAP Engineering & Construction Services,"333 S. Allison Pky, Suite 100",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
15063,ZAYALA FIELD SERVICES,ZAYALA FIELD SERVICES,NaN,NaN,NaN,NaN,303-549-5978,AZAVALA@ZAVALAFIELD SERVICES.COM,ZAYALA FIELD SERVICES,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%
15066,ZAYRA DIAZ,ZAYRA DIAZ,NaN,NaN,NaN,NaN,970-888-2605,ZDIAZ1820@GMAIL.COM,ZAYRA DIAZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Non,NON TAXABLE ITEM


### 3,183 row remaining.  The main_phone number column has to be cleaned and reformatted.  The first step will be to remove some of the text strings from the column.

In [10]:
# extract the phone number (digits only)
dfcl['main_phone'] = dfcl['main_phone'].str.extract('((?:\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4})', expand=True)
dfcl

<ipython-input-10-d905ad98591c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcl['main_phone'] = dfcl['main_phone'].str.extract('((?:\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4})', expand=True)


,customer,company,mr./ms./...,first_name,m.i.,last_name,main_phone,main_email,bill_to_1,bill_to_2,...,bill_to_4,bill_to_5,ship_to_1,ship_to_2,ship_to_3,ship_to_4,ship_to_5,terms,sales_tax_code,tax_item
0,JESUS SANCHEZ,JESUS SANCHEZ,NaN,NaN,NaN,NaN,670-342-6100,NaN,JESUS SANCHEZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tax,NON TAXABLE ITEM
2,1888 INDUSTRIAL SERVICES,1888 INDUSTRIAL SERVICES,NaN,NaN,NaN,NaN,970-702-7610,AP@1888IS.COM,1888 INDUSTRIAL SERVICES,"800 8TH AVE, SUITE 301",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
4,2 RINGS TRUCKING,2 RINGS TRUCKING,NaN,NaN,NaN,NaN,406-289-0901,NaN,2 RINGS TRUCKING,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Due on receipt,Non,NON TAXABLE ITEM
6,2 VALLEY BUILDERS,"2 VALLEY BUILDERS, INC",NaN,NaN,NaN,NaN,970-599-2134,2valleybuilders@gmail.com,"2 VALLEY BUILDERS, INC",6637 SPANISH BAY DRIVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
15,2020 APEX LLC,2020 APEX LLC,NaN,NaN,NaN,NaN,970.381.1081,RANDRE@SEARSREALESTATE.COM,2020 APEX LLC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15053,ZAK DIRT,"ZAK DIRT, INC.",NaN,BRANDI,NaN,WILSON,970-535-4657,BWILSON@ZAKDIRT.COM; accounting@zakdirt.com,"ZAK DIRT, INC.",14290 HILLTOP ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
15060,ZAP Engineering & Construction Services,ZAP Engineering & Construction Services,NaN,NaN,NaN,NaN,303-565-5567,apinvoices@zapecs.com,ZAP Engineering & Construction Services,"333 S. Allison Pky, Suite 100",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
15063,ZAYALA FIELD SERVICES,ZAYALA FIELD SERVICES,NaN,NaN,NaN,NaN,303-549-5978,AZAVALA@ZAVALAFIELD SERVICES.COM,ZAYALA FIELD SERVICES,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%
15066,ZAYRA DIAZ,ZAYRA DIAZ,NaN,NaN,NaN,NaN,970-888-2605,ZDIAZ1820@GMAIL.COM,ZAYRA DIAZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Non,NON TAXABLE ITEM


In [11]:
# remove period from main_phone column
# remove dashes from phone column
dfcl.main_phone= dfcl.main_phone.replace({'\.':''}, regex=True)
dfcl.main_phone= dfcl.main_phone.replace({'-': ''}, regex=True)
dfcl

C:\Users\Yaron\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,customer,company,mr./ms./...,first_name,m.i.,last_name,main_phone,main_email,bill_to_1,bill_to_2,...,bill_to_4,bill_to_5,ship_to_1,ship_to_2,ship_to_3,ship_to_4,ship_to_5,terms,sales_tax_code,tax_item
0,JESUS SANCHEZ,JESUS SANCHEZ,NaN,NaN,NaN,NaN,6703426100,NaN,JESUS SANCHEZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tax,NON TAXABLE ITEM
2,1888 INDUSTRIAL SERVICES,1888 INDUSTRIAL SERVICES,NaN,NaN,NaN,NaN,9707027610,AP@1888IS.COM,1888 INDUSTRIAL SERVICES,"800 8TH AVE, SUITE 301",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
4,2 RINGS TRUCKING,2 RINGS TRUCKING,NaN,NaN,NaN,NaN,4062890901,NaN,2 RINGS TRUCKING,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Due on receipt,Non,NON TAXABLE ITEM
6,2 VALLEY BUILDERS,"2 VALLEY BUILDERS, INC",NaN,NaN,NaN,NaN,9705992134,2valleybuilders@gmail.com,"2 VALLEY BUILDERS, INC",6637 SPANISH BAY DRIVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
15,2020 APEX LLC,2020 APEX LLC,NaN,NaN,NaN,NaN,9703811081,RANDRE@SEARSREALESTATE.COM,2020 APEX LLC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15053,ZAK DIRT,"ZAK DIRT, INC.",NaN,BRANDI,NaN,WILSON,9705354657,BWILSON@ZAKDIRT.COM; accounting@zakdirt.com,"ZAK DIRT, INC.",14290 HILLTOP ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
15060,ZAP Engineering & Construction Services,ZAP Engineering & Construction Services,NaN,NaN,NaN,NaN,3035655567,apinvoices@zapecs.com,ZAP Engineering & Construction Services,"333 S. Allison Pky, Suite 100",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
15063,ZAYALA FIELD SERVICES,ZAYALA FIELD SERVICES,NaN,NaN,NaN,NaN,3035495978,AZAVALA@ZAVALAFIELD SERVICES.COM,ZAYALA FIELD SERVICES,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%
15066,ZAYRA DIAZ,ZAYRA DIAZ,NaN,NaN,NaN,NaN,9708882605,ZDIAZ1820@GMAIL.COM,ZAYRA DIAZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Non,NON TAXABLE ITEM


In [12]:
# phone number column successfully cleaned
dfcl.main_phone.value_counts()

6013191527    3
2539735556    3
3603908885    2
7209330627    2
3036180511    2
             ..
9707372141    1
9705393307    1
9703711221    1
7203058183    1
7198592723    1
Name: main_phone, Length: 2902, dtype: int64

In [13]:
dfcl.main_phone.value_counts(sort=False)

7198592723    1
9703531389    1
9705844550    1
9709390174    1
9704024836    1
             ..
9706735431    1
7202986216    1
3039958726    1
9703054188    1
3039165086    1
Name: main_phone, Length: 2902, dtype: int64

### The customer column seems to be closely related to the company column.  I'm going to filter out rows where the two columns do not match and see what i can fix manually in a spreadsheet.  I will also create a seperate dataframe with clean rows and will append the two dfs.  

In [14]:
# filter for matching columns and store for later 
dfrop= dfcl.loc[(dfcl['company'] == dfcl['customer'])]
dfrop

,customer,company,mr./ms./...,first_name,m.i.,last_name,main_phone,main_email,bill_to_1,bill_to_2,...,bill_to_4,bill_to_5,ship_to_1,ship_to_2,ship_to_3,ship_to_4,ship_to_5,terms,sales_tax_code,tax_item
0,JESUS SANCHEZ,JESUS SANCHEZ,NaN,NaN,NaN,NaN,6703426100,NaN,JESUS SANCHEZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tax,NON TAXABLE ITEM
2,1888 INDUSTRIAL SERVICES,1888 INDUSTRIAL SERVICES,NaN,NaN,NaN,NaN,9707027610,AP@1888IS.COM,1888 INDUSTRIAL SERVICES,"800 8TH AVE, SUITE 301",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
4,2 RINGS TRUCKING,2 RINGS TRUCKING,NaN,NaN,NaN,NaN,4062890901,NaN,2 RINGS TRUCKING,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Due on receipt,Non,NON TAXABLE ITEM
15,2020 APEX LLC,2020 APEX LLC,NaN,NaN,NaN,NaN,9703811081,RANDRE@SEARSREALESTATE.COM,2020 APEX LLC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%
21,3 BEANS LLC,3 BEANS LLC,NaN,NaN,NaN,NaN,2539735556,3BEANSLLC@GMAIL.COM,3 BEANS LLC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15049,ZACH ROSATO,ZACH ROSATO,NaN,NaN,NaN,NaN,3038189442,NaN,ZACH ROSATO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,NON TAXABLE ITEM
15051,ZACH SNAVELY,ZACH SNAVELY,NaN,NaN,NaN,NaN,7207626709,NaN,ZACH SNAVELY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,NON TAXABLE ITEM
15060,ZAP Engineering & Construction Services,ZAP Engineering & Construction Services,NaN,NaN,NaN,NaN,3035655567,apinvoices@zapecs.com,ZAP Engineering & Construction Services,"333 S. Allison Pky, Suite 100",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
15063,ZAYALA FIELD SERVICES,ZAYALA FIELD SERVICES,NaN,NaN,NaN,NaN,3035495978,AZAVALA@ZAVALAFIELD SERVICES.COM,ZAYALA FIELD SERVICES,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%


In [15]:
# rows where customer and company name dont match
dfgc=dfcl.loc[(dfcl['company'] != dfcl['customer'])]
dfgc

,customer,company,mr./ms./...,first_name,m.i.,last_name,main_phone,main_email,bill_to_1,bill_to_2,...,bill_to_4,bill_to_5,ship_to_1,ship_to_2,ship_to_3,ship_to_4,ship_to_5,terms,sales_tax_code,tax_item
6,2 VALLEY BUILDERS,"2 VALLEY BUILDERS, INC",NaN,NaN,NaN,NaN,9705992134,2valleybuilders@gmail.com,"2 VALLEY BUILDERS, INC",6637 SPANISH BAY DRIVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
23,307 MEAT COMPANY,(307) MEAT COMPANY,NaN,NaN,NaN,NaN,3073439001,kelcey@307meat.com,(307) MEAT COMPANY,"3745 Cherrywood, E St.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,NON TAXABLE ITEM
24,38 NORTH CONSTRUCTION,38 NORTH CONSTRUCTION GROUP,NaN,JIM,NaN,HOPPER,7193589834,accountspayable@38northcg.com,38 NORTH CONSTRUCTION GROUP,"11641 Ridgeline Drive, Unit 160",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
33,4X INDUSTRIAL,"4X INDUSTRIAL, LLC",NaN,NaN,NaN,NaN,9703521790,ap@4xindustrial.com,"4X Industrial, LLC",800 8th Ave #300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
72,5280 S SERVICES,"5280 S SERVICES, LLC",NaN,FRANK,NaN,SILVA,9705186487,heather@5280sservices.com,"5280 S SERVICES, LLC",18494 CR 39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14996,WSP,LT ENVIRONMENTAL INC,NaN,NaN,NaN,NaN,3039625523,ap@ltenv.com,WSP USA,4600 WEST 60TH AVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 60,Non,NON TAXABLE ITEM
15004,XCEL ENERGY - Customer,XCEL ENERGY,NaN,NaN,NaN,NaN,NaN,Kami.R.Moore@xcelenergy.com,XCEL ENERGY,Attn: Robert McKay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Non,CREDIT CARD FEE 3%
15044,z GIENGERICH STRUCTURES DO NOT USE,GEINGERICH STRUCTURES,NaN,NaN,NaN,NaN,9702302052,will@giengerichstructures.com,GEINGERICH STRUCTURES,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Due on receipt,Non,NON TAXABLE ITEM
15053,ZAK DIRT,"ZAK DIRT, INC.",NaN,BRANDI,NaN,WILSON,9705354657,BWILSON@ZAKDIRT.COM; accounting@zakdirt.com,"ZAK DIRT, INC.",14290 HILLTOP ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM


In [16]:
#  create csv file for viewing in spreadsheet 
dfgc.to_csv('dfgc.csv')

### Notes on cleaning the customer column in spreadsheet:
#### - I would check with the db administrator before making any changes to the Customer sheet
#### - deleted "base rate sheet" from about 100 columns which left the company name in the column only  
#### - deleted remaining LLC, INC, Co from a total of about 10 rows
#### - kept rows marked "collections"
#### - most of the other non matching rows will be matched when they are in upper case

In [17]:
#  read the new partial dataframe ....413 clean customer rows
dfgcc = pd.read_excel('dfgcc.xlsx',index=True) 
dfgcc.to_csv('dfgcc.csv')
dfgcc

,Unnamed: 0,customer,company,mr./ms./...,first_name,m.i.,last_name,main_phone,main_email,bill_to_1,...,bill_to_4,bill_to_5,ship_to_1,ship_to_2,ship_to_3,ship_to_4,ship_to_5,terms,sales_tax_code,tax_item
0,6,2 VALLEY BUILDERS,"2 VALLEY BUILDERS, INC",NaN,NaN,NaN,NaN,9.705992e+09,2valleybuilders@gmail.com,"2 VALLEY BUILDERS, INC",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
1,23,307 MEAT COMPANY,(307) MEAT COMPANY,NaN,NaN,NaN,NaN,3.073439e+09,kelcey@307meat.com,(307) MEAT COMPANY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,NON TAXABLE ITEM
2,24,38 NORTH CONSTRUCTION,38 NORTH CONSTRUCTION GROUP,NaN,JIM,NaN,HOPPER,7.193590e+09,accountspayable@38northcg.com,38 NORTH CONSTRUCTION GROUP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
3,33,4X INDUSTRIAL,"4X INDUSTRIAL, LLC",NaN,NaN,NaN,NaN,9.703522e+09,ap@4xindustrial.com,"4X Industrial, LLC",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
4,72,5280 S SERVICES,"5280 S SERVICES, LLC",NaN,FRANK,NaN,SILVA,9.705186e+09,heather@5280sservices.com,"5280 S SERVICES, LLC",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,14996,WSP,LT ENVIRONMENTAL INC,NaN,NaN,NaN,NaN,3.039626e+09,ap@ltenv.com,WSP USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 60,Non,NON TAXABLE ITEM
409,15004,XCEL ENERGY - Customer,XCEL ENERGY,NaN,NaN,NaN,NaN,NaN,Kami.R.Moore@xcelenergy.com,XCEL ENERGY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Non,CREDIT CARD FEE 3%
410,15044,z GIENGERICH STRUCTURES DO NOT USE,GEINGERICH STRUCTURES,NaN,NaN,NaN,NaN,9.702302e+09,will@giengerichstructures.com,GEINGERICH STRUCTURES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Due on receipt,Non,NON TAXABLE ITEM
411,15053,ZAK DIRT,"ZAK DIRT, INC.",NaN,BRANDI,NaN,WILSON,9.705355e+09,BWILSON@ZAKDIRT.COM; accounting@zakdirt.com,"ZAK DIRT, INC.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM


In [18]:
# No duplicates in the new df
dfgcc[dfgcc["customer"].duplicated()] 

,Unnamed: 0,customer,company,mr./ms./...,first_name,m.i.,last_name,main_phone,main_email,bill_to_1,...,bill_to_4,bill_to_5,ship_to_1,ship_to_2,ship_to_3,ship_to_4,ship_to_5,terms,sales_tax_code,tax_item


### At this point I am going to append the new clean df to the previously filtered df.  The new appended df has 3,183 rows, and the customer column looks much cleaner and more closely related (for joining) to the customer column in the Resource sheet.

In [19]:
# append partial clean rows to filtered clean dfrop
dfm=dfrop.append(dfgcc) 
dfm

,customer,company,mr./ms./...,first_name,m.i.,last_name,main_phone,main_email,bill_to_1,bill_to_2,...,bill_to_5,ship_to_1,ship_to_2,ship_to_3,ship_to_4,ship_to_5,terms,sales_tax_code,tax_item,Unnamed: 0
0,JESUS SANCHEZ,JESUS SANCHEZ,NaN,NaN,NaN,NaN,6703426100,NaN,JESUS SANCHEZ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tax,NON TAXABLE ITEM,NaN
2,1888 INDUSTRIAL SERVICES,1888 INDUSTRIAL SERVICES,NaN,NaN,NaN,NaN,9707027610,AP@1888IS.COM,1888 INDUSTRIAL SERVICES,"800 8TH AVE, SUITE 301",...,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM,NaN
4,2 RINGS TRUCKING,2 RINGS TRUCKING,NaN,NaN,NaN,NaN,4062890901,NaN,2 RINGS TRUCKING,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Due on receipt,Non,NON TAXABLE ITEM,NaN
15,2020 APEX LLC,2020 APEX LLC,NaN,NaN,NaN,NaN,9703811081,RANDRE@SEARSREALESTATE.COM,2020 APEX LLC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%,NaN
21,3 BEANS LLC,3 BEANS LLC,NaN,NaN,NaN,NaN,2539735556,3BEANSLLC@GMAIL.COM,3 BEANS LLC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,WSP,LT ENVIRONMENTAL INC,NaN,NaN,NaN,NaN,3.03963e+09,ap@ltenv.com,WSP USA,4600 WEST 60TH AVE,...,NaN,NaN,NaN,NaN,NaN,NaN,Net 60,Non,NON TAXABLE ITEM,14996.0
409,XCEL ENERGY - Customer,XCEL ENERGY,NaN,NaN,NaN,NaN,NaN,Kami.R.Moore@xcelenergy.com,XCEL ENERGY,Attn: Robert McKay,...,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Non,CREDIT CARD FEE 3%,15004.0
410,z GIENGERICH STRUCTURES DO NOT USE,GEINGERICH STRUCTURES,NaN,NaN,NaN,NaN,9.7023e+09,will@giengerichstructures.com,GEINGERICH STRUCTURES,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Due on receipt,Non,NON TAXABLE ITEM,15044.0
411,ZAK DIRT,"ZAK DIRT, INC.",NaN,BRANDI,NaN,WILSON,9.70535e+09,BWILSON@ZAKDIRT.COM; accounting@zakdirt.com,"ZAK DIRT, INC.",14290 HILLTOP ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM,15053.0


#### 1st Join condition:
#### For each unique customer name (case insensitive) in the route sheet, create an object consisting of the following fields: ○ From the customer sheet:
- Customer
- Bill to 1
-  Bill to 2
- Bill to 3
- Main Phone
- Note: each phone number should be formatted to include only numeric characters. For example “555-123-4567: Tracy” should become “5551234567”.
- Main Email
- Terms

In [20]:
dfr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2528 entries, 0 to 2527
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer            2528 non-null   object
 1   TOILET #            2528 non-null   object
 2   Job Name/Well Name  1077 non-null   object
 3   Address             2384 non-null   object
 4   City                2523 non-null   object
 5   Schedule            2528 non-null   object
 6   Scheduled Day       2528 non-null   object
 7   Charge Type         2517 non-null   object
dtypes: object(8)
memory usage: 158.1+ KB


In [21]:
# remove white space
dfr.Customer.str.strip()

0                         src
1          Hauer Custom Homes
2       Ridgeway Custom homes
3                 ASTER RIDGE
4                  Blackeagle
                ...          
2523          BLUE BEAR WASTE
2524               BEAR CREEK
2525    TRISTAR HEATING & AIR
2526          MARTIN MARIETTA
2527          MARTIN MARIETTA
Name: Customer, Length: 2528, dtype: object

In [22]:
#  remove everything after the dashes in the column 
dfr.Customer.str.split('\n').str[0]

0                         src
1          Hauer Custom Homes
2       Ridgeway Custom homes
3                 ASTER RIDGE
4                  Blackeagle
                ...          
2523          BLUE BEAR WASTE
2524               BEAR CREEK
2525    TRISTAR HEATING & AIR
2526          MARTIN MARIETTA
2527          MARTIN MARIETTA
Name: Customer, Length: 2528, dtype: object

In [23]:
# 1,706 duplicate rows in the resource sheet
dfr.duplicated('Customer').sum()

1706

In [51]:
#  unique names of the Customer column in the route sheet and store in new df
dfru=dfr["Customer"].unique() 

In [25]:
# convert array to pandas dataframe
dfrc = pd.DataFrame(dfru, columns = ["customer"])

print(dfrc)
print(type(dfrc))

                  customer
0                      src
1       Hauer Custom Homes
2    Ridgeway Custom homes
3              ASTER RIDGE
4               Blackeagle
..                     ...
817         INTEGRITY GOLF
818    GARNEY CONSTRUCTION
819     CLEAR CREEK (CCRP)
820            MARTY FRANK
821  TRISTAR HEATING & AIR

[822 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


####  Finally time to join the 822 unique customer values (dfrc) from the Route sheet to the customers in the clean Customers (dfm, 3,183 rows).  I'm going to change both columns to upper case first and remove LLC and INC from the columns.

In [26]:
#remove LLC and INC
dfrc.customer = dfrc.customer.str.replace('LLC/','')
dfrc.customer = dfrc.customer.str.replace('INC/','')
dfrc

,customer
0,src
1,Hauer Custom Homes
2,Ridgeway Custom homes
3,ASTER RIDGE
4,Blackeagle
...,...
817,INTEGRITY GOLF
818,GARNEY CONSTRUCTION
819,CLEAR CREEK (CCRP)
820,MARTY FRANK


In [27]:
# convert both columns to Upper Case
dfm.customer = dfm.customer.str.upper()
dfrc.customer = dfrc.customer.str.upper()
dfrc

,customer
0,SRC
1,HAUER CUSTOM HOMES
2,RIDGEWAY CUSTOM HOMES
3,ASTER RIDGE
4,BLACKEAGLE
...,...
817,INTEGRITY GOLF
818,GARNEY CONSTRUCTION
819,CLEAR CREEK (CCRP)
820,MARTY FRANK


In [28]:
df_clean = pd.merge(dfm, dfrc,  on=['customer'])
df_clean

,customer,company,mr./ms./...,first_name,m.i.,last_name,main_phone,main_email,bill_to_1,bill_to_2,...,bill_to_5,ship_to_1,ship_to_2,ship_to_3,ship_to_4,ship_to_5,terms,sales_tax_code,tax_item,Unnamed: 0
0,3 BEANS LLC,3 BEANS LLC,NaN,NaN,NaN,NaN,2539735556,3BEANSLLC@GMAIL.COM,3 BEANS LLC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%,NaN
1,3M FARMS,3M FARMS,NaN,NaN,NaN,NaN,7203533101,3MFARMSERIE@GMAIL.COM,3M FARMS,4406 CR 3,...,NaN,NaN,NaN,NaN,NaN,NaN,Due on receipt,Non,NON TAXABLE ITEM,NaN
2,5 BELL RANCH,5 BELL RANCH,NaN,NaN,NaN,NaN,7202013780,5BELLRANCH1@GMAIL.COM,5 BELL RANCH,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CREDIT CARD ONLY,Tax,CREDIT CARD FEE 3%,NaN
3,5 STAR INDUSTRIES,5 STAR INDUSTRIES,NaN,VICKI,NaN,ALVAREZ,9703780140,5starapar@gmail.com,5 STAR INDUSTRIES,701 5TH STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,Due on receipt,Non,NON TAXABLE ITEM,NaN
4,A1 ORGANICS,A1 Organics,NaN,NaN,NaN,NaN,9704543492,colleenshepherd@a1organics.com,"Lambland, Inc DBA: A-1 Organics",16350 WCR 76,...,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,VICTORY UNLIMITED CONSTRUCTION,"VICTORY UNLIMITED CONSTRUCTION, LLC",NaN,NaN,NaN,NaN,3.1786e+09,KKhan@victoryuc.com,"VICTORY UNLIMITED CONSTRUCTION, LLC",6831 E. 32ND ST. STE. #300,...,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Tax,NON TAXABLE ITEM,14261.0
344,WARD CONSTRUCTION,"WARD CONSTRUCTION, CO.",NaN,NaN,NaN,NaN,3.0379e+09,team@ward-construction.com,"WARD CONSTRUCTION, CO.",2750 S. SHOSHONE ST STE#315,...,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM,14355.0
345,WESTERN CONSTRUCTION,"WESTERN CONSTRUCTION MANAGEMENT, INC.",NaN,NaN,NaN,NaN,9.70305e+09,RACHEL@WCMBUILDINGS.COM,"WESTERN CONSTRUCTION MANAGEMENT, INC.",724 SOARING EAGLE DRIVE,...,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM,14525.0
346,ZAK DIRT,"ZAK DIRT, INC.",NaN,BRANDI,NaN,WILSON,9.70535e+09,BWILSON@ZAKDIRT.COM; accounting@zakdirt.com,"ZAK DIRT, INC.",14290 HILLTOP ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,Net 30,Non,NON TAXABLE ITEM,15053.0


In [29]:
#  list the columns of the merged df
df_clean.columns.tolist()

['customer',
 'company',
 'mr./ms./...',
 'first_name',
 'm.i.',
 'last_name',
 'main_phone',
 'main_email',
 'bill_to_1',
 'bill_to_2',
 'bill_to_3',
 'bill_to_4',
 'bill_to_5',
 'ship_to_1',
 'ship_to_2',
 'ship_to_3',
 'ship_to_4',
 'ship_to_5',
 'terms',
 'sales_tax_code',
 'tax_item',
 'Unnamed: 0']

In [30]:
# drop columns
df_clean = df_clean.drop(['bill_to_4','bill_to_5',
 'ship_to_1',
 'ship_to_2',
 'ship_to_3',
 'ship_to_4',
 'ship_to_5',
 'sales_tax_code',
 'tax_item','mr./ms./...',
 'first_name',
 'm.i.',
 'last_name','Unnamed: 0'], axis=1)

In [31]:
# merged df has 348 rows and 8 columns
print(df_clean.shape)
df_clean.head()

(348, 8)


,customer,company,main_phone,main_email,bill_to_1,bill_to_2,bill_to_3,terms
0,3 BEANS LLC,3 BEANS LLC,2539735556,3BEANSLLC@GMAIL.COM,3 BEANS LLC,NaN,NaN,CREDIT CARD ONLY
1,3M FARMS,3M FARMS,7203533101,3MFARMSERIE@GMAIL.COM,3M FARMS,4406 CR 3,"ERIE, CO 80516",Due on receipt
2,5 BELL RANCH,5 BELL RANCH,7202013780,5BELLRANCH1@GMAIL.COM,5 BELL RANCH,NaN,NaN,CREDIT CARD ONLY
3,5 STAR INDUSTRIES,5 STAR INDUSTRIES,9703780140,5starapar@gmail.com,5 STAR INDUSTRIES,701 5TH STREET,"GREELEY, CO 80631",Due on receipt
4,A1 ORGANICS,A1 Organics,9704543492,colleenshepherd@a1organics.com,"Lambland, Inc DBA: A-1 Organics",16350 WCR 76,"Eaton, CO 80615",Net 30


In [32]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 348 entries, 0 to 347
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customer    348 non-null    object
 1   company     345 non-null    object
 2   main_phone  338 non-null    object
 3   main_email  327 non-null    object
 4   bill_to_1   348 non-null    object
 5   bill_to_2   168 non-null    object
 6   bill_to_3   166 non-null    object
 7   terms       347 non-null    object
dtypes: object(8)
memory usage: 24.5+ KB


#### View df_clean in a spreadsheet

In [33]:
%%script false --no-raise-error
df_clean.to_csv('df_clean.csv')

In [34]:
# double check these names in the spreadsheet...ok to drop and keep first
df_clean.customer.value_counts()

T3 INCORPORATED                2
GARNEY CONSTRUCTION            2
STERLING ENERGY INVESTMENTS    2
MARTIN MARIETTA                2
EAGLE CONSTRUCTION             2
                              ..
BRENDA FOLIZ                   1
TREVA STRAIN                   1
MARCIA BECKER                  1
KENNY PLATTE CUTTING HORSES    1
OSCAR MENDEZ                   1
Name: customer, Length: 336, dtype: int64

In [35]:
# drop duplicates, 336 rows remaining
df_clean=df_clean.drop_duplicates(subset='customer', keep='first', inplace=False)
df_clean

,customer,company,main_phone,main_email,bill_to_1,bill_to_2,bill_to_3,terms
0,3 BEANS LLC,3 BEANS LLC,2539735556,3BEANSLLC@GMAIL.COM,3 BEANS LLC,NaN,NaN,CREDIT CARD ONLY
1,3M FARMS,3M FARMS,7203533101,3MFARMSERIE@GMAIL.COM,3M FARMS,4406 CR 3,"ERIE, CO 80516",Due on receipt
2,5 BELL RANCH,5 BELL RANCH,7202013780,5BELLRANCH1@GMAIL.COM,5 BELL RANCH,NaN,NaN,CREDIT CARD ONLY
3,5 STAR INDUSTRIES,5 STAR INDUSTRIES,9703780140,5starapar@gmail.com,5 STAR INDUSTRIES,701 5TH STREET,"GREELEY, CO 80631",Due on receipt
4,A1 ORGANICS,A1 Organics,9704543492,colleenshepherd@a1organics.com,"Lambland, Inc DBA: A-1 Organics",16350 WCR 76,"Eaton, CO 80615",Net 30
...,...,...,...,...,...,...,...,...
343,VICTORY UNLIMITED CONSTRUCTION,"VICTORY UNLIMITED CONSTRUCTION, LLC",3.1786e+09,KKhan@victoryuc.com,"VICTORY UNLIMITED CONSTRUCTION, LLC",6831 E. 32ND ST. STE. #300,"INDIANAPOLIS, IN 46226",Net 30
344,WARD CONSTRUCTION,"WARD CONSTRUCTION, CO.",3.0379e+09,team@ward-construction.com,"WARD CONSTRUCTION, CO.",2750 S. SHOSHONE ST STE#315,"ENGLEWOOD, CO 80110",Net 30
345,WESTERN CONSTRUCTION,"WESTERN CONSTRUCTION MANAGEMENT, INC.",9.70305e+09,RACHEL@WCMBUILDINGS.COM,"WESTERN CONSTRUCTION MANAGEMENT, INC.",724 SOARING EAGLE DRIVE,"LaPorte, CO 80535",Net 30
346,ZAK DIRT,"ZAK DIRT, INC.",9.70535e+09,BWILSON@ZAKDIRT.COM; accounting@zakdirt.com,"ZAK DIRT, INC.",14290 HILLTOP ROAD,"LONGMONT, CO. 80504",Net 30


### Time to attach an array created from the route sheet
- An array of jobs, each with the fields:
- Job Name/Well Name,
- Address
- City
- Schedule
- Scheduled Day
- Charge Type
- TOILET #

In [36]:
#  have to joing df_clean (array, 336 rows, 'customer column', CAPS) with dfr(2,528 rows, Customer, non caps) with jobs array (no customer)

In [37]:
dfr

,Customer,TOILET #,Job Name/Well Name,Address,City,Schedule,Scheduled Day,Charge Type
0,src,1,Ag Pad FRAC,O Street x 59th Ave W1.1 S into,West Greeley,2xWeekly,7/22/18change billing to production,pending
1,Hauer Custom Homes,3611,NaN,"19299 CR 70, Eaton",Eaton,Weekly,address does not exist 10/7/19,MONTHLY (8/1/19)
2,Ridgeway Custom homes,1,NaN,"6879 Crooked Stick, Windsor",West Windsor,Weekly,address does not exist 3/22/19,MONTHLY (9/19/18)
3,ASTER RIDGE,3051,NaN,1827 AA ST,EAST GREELEY,WEEKLY,BISON RIDGE TOOK OVER TOILET 1/27/22,MONTHLY (10/12/21)
4,Blackeagle,"3540, 2489",Angus Compressor Station,"60315 CR 71, Grover\n128x69 E1 N into",Grover,Weekly,BLOWNOVER/TRADED 10/24/19,MONTHLY (6/11/19)
...,...,...,...,...,...,...,...,...
2523,BLUE BEAR WASTE,4431,NaN,7789 W 5TH AVE,LAKEWOOD,WEEKLY,WEDNESDAY,MONTHLY (5/20/22)
2524,BEAR CREEK,4891,NaN,1074 MARFELL ST,ERIE,WEEKLY,TUESDAY,MONTHLY (5/20/22)
2525,TRISTAR HEATING & AIR,"4391, 4392",NaN,21350 CR 10,HUDSON,WEEKLY,FRIDAY,MONTHLY (5/20/22)
2526,MARTIN MARIETTA,TRAILER 1216,NaN,NaN,NORTH FORT COLLINS,2XWEEKLY,TUESDAY/FRIDAY,MONTHLY (5/20/22)


In [38]:
dfr['Customer'] = dfr['Customer'].str.upper()
dfr

,Customer,TOILET #,Job Name/Well Name,Address,City,Schedule,Scheduled Day,Charge Type
0,SRC,1,Ag Pad FRAC,O Street x 59th Ave W1.1 S into,West Greeley,2xWeekly,7/22/18change billing to production,pending
1,HAUER CUSTOM HOMES,3611,NaN,"19299 CR 70, Eaton",Eaton,Weekly,address does not exist 10/7/19,MONTHLY (8/1/19)
2,RIDGEWAY CUSTOM HOMES,1,NaN,"6879 Crooked Stick, Windsor",West Windsor,Weekly,address does not exist 3/22/19,MONTHLY (9/19/18)
3,ASTER RIDGE,3051,NaN,1827 AA ST,EAST GREELEY,WEEKLY,BISON RIDGE TOOK OVER TOILET 1/27/22,MONTHLY (10/12/21)
4,BLACKEAGLE,"3540, 2489",Angus Compressor Station,"60315 CR 71, Grover\n128x69 E1 N into",Grover,Weekly,BLOWNOVER/TRADED 10/24/19,MONTHLY (6/11/19)
...,...,...,...,...,...,...,...,...
2523,BLUE BEAR WASTE,4431,NaN,7789 W 5TH AVE,LAKEWOOD,WEEKLY,WEDNESDAY,MONTHLY (5/20/22)
2524,BEAR CREEK,4891,NaN,1074 MARFELL ST,ERIE,WEEKLY,TUESDAY,MONTHLY (5/20/22)
2525,TRISTAR HEATING & AIR,"4391, 4392",NaN,21350 CR 10,HUDSON,WEEKLY,FRIDAY,MONTHLY (5/20/22)
2526,MARTIN MARIETTA,TRAILER 1216,NaN,NaN,NORTH FORT COLLINS,2XWEEKLY,TUESDAY/FRIDAY,MONTHLY (5/20/22)


### I'm going to create a SQL database, where I will join the columns from the Resource table to the clean Customer table using the cutomer column as the matching field.  

In [39]:
#  creat SQL lite database 
conn = sqlite3.connect('servicepro.sqlite')

In [40]:
%%script false --no-raise-error
# frames to SQL db
df_clean.to_sql("clq",conn)
dfr.to_sql('resq',conn)

In [41]:
# SQL join for array of jobs
dfjbs= pd.read_sql("SELECT c.customer,r.[Job Name/Well Name],r.Address, r.City, r.Schedule, r.[Scheduled Day], r.[Charge Type], r.[Toilet #]  FROM 'clq' c JOIN 'resq' r ON c.customer= r.Customer ",conn)
dfjbs

,customer,Job Name/Well Name,Address,City,Schedule,Scheduled Day,Charge Type,TOILET #
0,MARTIN MARIETTA,None,None,NORTH FORT COLLINS,2XWEEKLY,TUESDAY/FRIDAY,MONTHLY (5/20/22),TRAILER 1216
1,MARTIN MARIETTA,None,None,GREELEY,2XWEEKLY,MONDAY/THURSDAY,MONTHLY (5/18/21),TRAILER 1281
2,MARTIN MARIETTA,None,None,WINDSOR,WEEKLY,TRADED OUT 6/18/21,MONTHLY (5/21/21),TRAILER 1312
3,MARTIN MARIETTA,None,None,GREELEY,WEEKLY,MONDAY/THURSDAY,MONTHLY (6/18/21),TRAILER 1314
4,MARTIN MARIETTA,None,None,NORTH FORT COLLINS,2XWEEKLY,TUESDAY/FRIDAY,MONTHLY (5/20/22),TRAILER 1321
...,...,...,...,...,...,...,...,...
914,BRINKMAN CONSTRUCTION,None,2525 71ST AVE,WEST GREELEY,WEEKLY,WEDNESDAY,MONTHLY (12/6/21) (4/26/22)-1,"2510, 4575"
915,MISIJA CONSTRUCTION,None,1412 60TH AVE,WEST GREELEY,WEEKLY,WEDNESDAY,MONTHLY (3/9/22),2392
916,SEIBER DAIRY,None,18374 CR 2,WIGGINS,WEEKLY,WEDNESDAY,MONTHLY (10/6/21),2957
917,SKY RIDGE DEVELOPMENT,None,118 7TH AVE,WIGGINS,WEEKLY,WEDNESDAY,MONTHLY (3/4/22),5012


#### The frame above (dfjbs) contains clean customer names matched to their corresponding job route.  Time to create an array with each row representing a route that will be displayed when the customer name is selected.    

In [47]:
# create jobs array with fields 
jobs = dfjbs[['Job Name/Well Name', 'Address', 'City', 'Schedule',
       'Scheduled Day', 'Charge Type', 'TOILET #']]
jobs.to_numpy()

array([[None, None, 'NORTH FORT COLLINS', ..., 'TUESDAY/FRIDAY',
        'MONTHLY (5/20/22)', 'TRAILER 1216'],
       [None, None, 'GREELEY', ..., 'MONDAY/THURSDAY',
        'MONTHLY (5/18/21)', 'TRAILER 1281'],
       [None, None, 'WINDSOR', ..., 'TRADED OUT 6/18/21',
        'MONTHLY (5/21/21)', 'TRAILER 1312'],
       ...,
       [None, '18374 CR 2', 'WIGGINS', ..., 'WEDNESDAY',
        'MONTHLY (10/6/21)', '2957'],
       [None, '118 7TH AVE', 'WIGGINS', ..., 'WEDNESDAY',
        'MONTHLY (3/4/22)', '5012'],
       [None, '7777 W 29TH AVE', 'WHEAT RIDGE', ..., 'WEDNESDAY',
        'MONTHLY (5/14/22)', 'TRAILER 1211 (SINGLE)']], dtype=object)

####  Open sample JSON that was provided and view in dataframe 

In [43]:
# Opening Sample JSON file 
f = open('Sample Output - ServiceCore Data Test.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

for i in data[1]:
    print(i)
  
    
f.close()

Customer
Bill to 1
Bill to 2
Bill to 3
Main Phone
Main Email
Terms
jobs


In [48]:
dfsa=pd.json_normalize(data, max_level=0)
dfsa

,Customer,Bill to 1,Bill to 2,Bill to 3,Main Phone,Main Email,Terms,jobs
0,COLORADO POND PROS,COLORADO POND PROS,None,None,3037041505,COLORADOPONDPROS@GMAIL.COM,CREDIT CARD ONLY,"[{'Job Name/Well Name': None, 'Address': '8450..."
1,GERRARD EXCAVATING,"GERRARD EXCAVATING, INC.",27154 COUNTY RD 13,"JOHNSTOWN, CO 80534",9706691463,invoices.gerrardex@gmail.com,Net 30,[{'Job Name/Well Name': 'NORTHRIDGE IRRIGATION...


#### Convert the new combined df to json that can be passed into API

In [49]:
dfjz= jobs.to_json(orient='columns')

In [50]:
##  view df in json
df_clean['jobs']= dfjz
df_clean

,customer,company,main_phone,main_email,bill_to_1,bill_to_2,bill_to_3,terms,jobs
0,3 BEANS LLC,3 BEANS LLC,2539735556,3BEANSLLC@GMAIL.COM,3 BEANS LLC,NaN,NaN,CREDIT CARD ONLY,"{""Job Name\/Well Name"":{""0"":null,""1"":null,""2"":..."
1,3M FARMS,3M FARMS,7203533101,3MFARMSERIE@GMAIL.COM,3M FARMS,4406 CR 3,"ERIE, CO 80516",Due on receipt,"{""Job Name\/Well Name"":{""0"":null,""1"":null,""2"":..."
2,5 BELL RANCH,5 BELL RANCH,7202013780,5BELLRANCH1@GMAIL.COM,5 BELL RANCH,NaN,NaN,CREDIT CARD ONLY,"{""Job Name\/Well Name"":{""0"":null,""1"":null,""2"":..."
3,5 STAR INDUSTRIES,5 STAR INDUSTRIES,9703780140,5starapar@gmail.com,5 STAR INDUSTRIES,701 5TH STREET,"GREELEY, CO 80631",Due on receipt,"{""Job Name\/Well Name"":{""0"":null,""1"":null,""2"":..."
4,A1 ORGANICS,A1 Organics,9704543492,colleenshepherd@a1organics.com,"Lambland, Inc DBA: A-1 Organics",16350 WCR 76,"Eaton, CO 80615",Net 30,"{""Job Name\/Well Name"":{""0"":null,""1"":null,""2"":..."
...,...,...,...,...,...,...,...,...,...
343,VICTORY UNLIMITED CONSTRUCTION,"VICTORY UNLIMITED CONSTRUCTION, LLC",3.1786e+09,KKhan@victoryuc.com,"VICTORY UNLIMITED CONSTRUCTION, LLC",6831 E. 32ND ST. STE. #300,"INDIANAPOLIS, IN 46226",Net 30,"{""Job Name\/Well Name"":{""0"":null,""1"":null,""2"":..."
344,WARD CONSTRUCTION,"WARD CONSTRUCTION, CO.",3.0379e+09,team@ward-construction.com,"WARD CONSTRUCTION, CO.",2750 S. SHOSHONE ST STE#315,"ENGLEWOOD, CO 80110",Net 30,"{""Job Name\/Well Name"":{""0"":null,""1"":null,""2"":..."
345,WESTERN CONSTRUCTION,"WESTERN CONSTRUCTION MANAGEMENT, INC.",9.70305e+09,RACHEL@WCMBUILDINGS.COM,"WESTERN CONSTRUCTION MANAGEMENT, INC.",724 SOARING EAGLE DRIVE,"LaPorte, CO 80535",Net 30,"{""Job Name\/Well Name"":{""0"":null,""1"":null,""2"":..."
346,ZAK DIRT,"ZAK DIRT, INC.",9.70535e+09,BWILSON@ZAKDIRT.COM; accounting@zakdirt.com,"ZAK DIRT, INC.",14290 HILLTOP ROAD,"LONGMONT, CO. 80504",Net 30,"{""Job Name\/Well Name"":{""0"":null,""1"":null,""2"":..."
